In [ ]:
import json
import boto3
import uuid
from datetime import datetime, timedelta
import os
import time
import re
import csv

#Client
athena = boto3.client('athena')
s3 = boto3.client('s3')

#Env
output_bucket = os.environ['Athena_Output']
chunk = os.environ['chunk']
upload_bucket = os.environ['BUCKET_NAME']

#Empty List
query_id_list = []

#Yesterday Date                                             
yesterday = datetime.now() - timedelta(days=1)

#Main Function
def lambda_handler(event, context):
    
    #SQL Query
    sql_query = '''
       with clordtrail_log AS (
            SELECT
            json_extract(responseelements,
            '$.queryExecutionId') AS query_id,
            from_iso8601_timestamp(eventtime) AS datetime
            FROM    
            WHERE   eventsource='athena.amazonaws.com'
            AND eventname='StartQueryExecution'
            AND json_extract(responseelements, '$.queryExecutionId') is NOT null)
            SELECT *
            FROM   clordtrail_log
            WHERE  datetime > date_add('day',-1,now() )'''
    
    #Partition
    s3_output = 's3://{}/{}/{}/{}/'.format(output_bucket,yesterday.year,yesterday.month,yesterday.day)  
    
    #Athena Query
    batch_query_id = run_query(sql_query,s3_output, max_execution=5)
    
    #batch_get_query_execution limited 50 query id
    #Split List into chunk 
    batch_query_chunks = list(divide_chunks(batch_query_id, 50))
    
    batch_query(batch_query_chunks)
    
def run_query(query, s3_output, max_execution=5):
    
    #Run SQL Query
    response = athena.start_query_execution(
                                    QueryString=query,
                                    ResultConfiguration={
                                    'OutputLocation': s3_output
                                                        }
                                                )

    #Query ID
    execution_id = response['QueryExecutionId']
    
    print("QueryExecutionId = " + str(execution_id))
    
    state  = 'QUEUED'

    while (max_execution > 0 and state in ['RUNNING', 'QUEUED']):
        max_execution = max_execution - 1
        print("maxexecution=" + str(max_execution))
        response = athena.get_query_execution(QueryExecutionId=execution_id)  

        if 'QueryExecution' in response and \
                'Status' in response['QueryExecution'] and \
                'State' in response['QueryExecution']['Status']:

                state = response['QueryExecution']['Status']['State']
                print(state)
                
                if state == 'SUCCEEDED':
                    results = athena.get_query_results(QueryExecutionId=execution_id,
                                                                MaxResults=1000)  
                                                                
                    for i in range(1,len(results['ResultSet']['Rows'])):
                        query_id_list.extend(re.findall(r'"([^"]*)"',results['ResultSet']['Rows'][i]['Data'][0].get('VarCharValue')))
                        
                    #print(query_id_list)
    
                    print(query_id_list[0:5]) 
                     
                elif state == 'FAILED' or state == 'CANCELLED':
                    return False
                    
        time.sleep(30)
    
    return query_id_list
    
#Function about split list into chunk
def divide_chunks(batch_query_list:list, chunk): 

    for i in range(0, len(batch_query_list), chunk):  
        yield batch_query_list[i:i + chunk] 
  
    
def batch_query(batch_query_chunks):
    
    for j in range(0,len(batch_query_chunks)):
        response = athena.batch_get_query_execution(
                                    QueryExecutionIds=batch_query_chunks[j]   
                                                )

        f = open("/tmp/csv_file.csv", "w+")
        temp_csv_file = csv.writer(f) 
        temp_csv_file.writerow(["queryExecutionId","QueryStatement", "DataScannedInBytes", "TotalExecutionTimeInMillis" ])
        
        for i in range(len(response['QueryExecutions'])):
            temp_csv_file.writerow(
                                    [
                            response['QueryExecutions'][i]['QueryExecutionId'],
                            response['QueryExecutions'][i]['Query'],
                            response['QueryExecutions'][i]['Statistics']['DataScannedInBytes'],
                            response['QueryExecutions'][i]['Statistics']['TotalExecutionTimeInMillis']
                                    ]
                                    )
        f.close()            
                                    
        s3.upload_file('/tmp/csv_file.csv', upload_bucket,'{}-{}-{}.csv'.format(yesterday.year,yesterday.month,yesterday.day))
                            
            
            
        
    return False
    

    
    